# DLfsP SimpleRNN am Beispiel PSF
Kurs Deep Learning für sequentielle Prozessdaten

#### In diesem Notebook wird ein einfaches Rekurrentes Neuronales Netz mit TensorFlow auf Zeitreihendaten einer Profilschienenführung (PSF) angewendet. 

Linearführungen und Profilschienenführungen sind für einen nicht unbedeutenden Teil der Ausfälle bei Werkzeugmaschinen.
Typische Fehlerfälle die dabei auftreten sind Mangelschmierungen, Pittings an Laufbahnen oder an den Wälzkörpern. In diesem Beispiel gibt es Daten von Normalfahrten und vom Zustand Pitting.  

Während der Versuche wurde mittels einem 3-achsigen MEMS-Sensor Daten vom Wagen aufgenommen. 
Für diese Aufgabe wurde nur die Beschleunigung in Verfahrrichtung (Acc_X) verwendet. Ziel ist die Erkennung, also Klassifikation nach dem Zustand ('Status').  


### Data-Mining-Prozess:
![Daten erfassen -> Daten erkunden -> Daten vorbereiten -> Modelle bilden -> Modelle validieren -> Modell testen & anwenden](Prozess_Modellentwicklung_v2.png "model development")

## 0. Bibliotheken importieren

In [ ]:
''' Import der wichtigsten Pakete. '''
import numpy as np
import pandas as pd
import random 
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import seaborn as sns

#Einstellungen für die Grafikausgabe
style = 'seaborn-whitegrid'
plt.style.use(style)
plt.rcParams.update({'font.size': 14})  # Schriftgröße aller Textzeichen im Graphen


In [ ]:
''' Überprüfe, ob TensorFlow installiert ist und die GPU oder die CPU nutzt. '''
print('TF version: ' + tf.__version__)
if tf.test.is_built_with_gpu_support():
    if len(tf.config.list_physical_devices('GPU'))==0:
         print('Achtung, TensorFlow nutzt gerade die CPU und nicht die GPU!')
    else:
        print('Default GPU Device: {}'.format(tf.config.list_physical_devices('GPU')))
        # GPU-Memory-Management:
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        session = tf.compat.v1.Session(config=config)
else:
    print('TensorFlow CPU version active')

TODO:
* Wähle eine Zahl zwischen 1 und 100 für die Generierung deiner spezifischen Zufallszahlen my_seed=

Damit Ergebnisse Reproduzierbar sind, müssen mehrere Seeds für Zufallszahlengeneratoren gesetzt werden. 

AUSGABE:
* Gewählte Zufallszahl

In [ ]:
''' Setzen von Seeds, um Reproduzierbarkeit zu ermöglichen. '''
# Erstelle eigene Zufallszahlen
my_seed = TODO

# Ausgabe gewählte Zufallszahlen
print("\nGewählte Zahl für Zufallszahlen: \t" + str(my_seed))

# Seeds für diverse Zufallszahlengeneratoren setzen 
os.environ['PYTHONHASHSEED']=str(my_seed)
tf.random.set_seed(my_seed)
random.seed(my_seed)
np.random.seed(my_seed)

## 1. Daten erfassen

In [ ]:
df = pd.read_csv('PSF_Pitting_kurz.csv') 
print('Daten erfolgreich importiert')

## 2. Daten erkunden

In [ ]:
""" ersten Datensätze als Beispiel"""
df.head(5)

In [ ]:
""" Statistische Beschreibung Datensatz"""
df.describe()

In [ ]:
# Einige Zyklen plotten
cycles=[]
cycles = random.sample(df['run_ID'].drop_duplicates().to_list(),3)

plt.figure(figsize=(12, 8))
plt.ylabel('Beschleunigung in X')
for c in cycles:
    status = df[df['run_ID']==c].reset_index().loc[0,'Status']
    plt.plot(df[df['run_ID']==c]['time_ms'],df[df['run_ID']==c]['Acc_X'],label='Run '+str(c) + ' Status ' + str(status))

plt.xlabel('Zeit in ms')
plt.legend()
plt.show()

In [ ]:
'''Berechnung der No Information Rate (NIR)'''
# Anzahl der Fälle je Klasse
print('Klasse 0:\t' + str(len(df[df['Status']==0]['run_ID'].unique())))
print('Klasse 1:\t' + str(len(df[df['Status']==1]['run_ID'].unique())))

# No Information Rate:
# Dumme Schätzung, immer größte Klasse
# NIR = Anzahl Beispiele größte Klasse / Anzahl Beispiele gesamt

print('NIR:\t\t' + str(round(len(df[df['Status']==0]['run_ID'].unique()) / len(df['run_ID'].unique()), 4)))

## 3. Daten vorbereiten

In [ ]:
""" Aufteilung in X und y"""
# Maximale Zykluslänge
maxlen = df['run_ID'].value_counts().max()    

# Aufteilung X und y Daten als DataFrames
y_df = df[['Status','run_ID']]
X_df = df.reset_index(drop=True)

In [ ]:
""" Umwandeln der DataFrames in numpy arrays und group by run_ID"""
g = X_df.groupby('run_ID').cumcount()
X = (X_df.set_index(['run_ID',g])
     .unstack()
     .stack().groupby(level=0)
     .apply(lambda x: x['Acc_X'].values)
     .to_numpy())
g = y_df.groupby('run_ID').cumcount()
y = (y_df.set_index(['run_ID',g])
     .unstack()
     .stack().groupby(level=0)
     .apply(lambda x: np.rint(x.sum()/len(x)))
     .to_numpy().astype("int32"))

print('Shapes der numpy Arrays X und y:')
print(X.shape, y.shape)

In [ ]:
'''Auffüllen der ungleichen Messreihen'''
# Auffüllen mit führenden Nullen, damit die Zeitreihen die gleiche Länge haben
X_pre = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=maxlen, padding='pre', dtype="float32")

In [ ]:
""" Aufteilen in Trainings-, Validierungs und Testdaten. """
# Festlegen des Anteils an Test- und Validierungsdaten
test_split = 0.2
validation_split = 0.2

# Aufteilung der Daten
# Testdaten abspalten# Aufteilung der Daten
X_train, X_test, y_train, y_test = train_test_split(X_pre, y, 
                                                    test_size=test_split, 
                                                    shuffle=True, 
                                                    random_state=my_seed)

# Aufsplitten in Trainings- und Validierungsdaten
X_train, X_val, y_train ,y_val = train_test_split(X_train, y_train, 
                                                  test_size=validation_split, 
                                                  shuffle=True, 
                                                  random_state=my_seed)

nr_train=len(X_train)
nr_val=len(X_val)
nr_test=len(X_test)
print(nr_train, "Training sequences")
print(nr_val, "Validation sequences")
print(nr_test, "Test sequences")

In [ ]:
""" Daten normalisieren. """
# Importieren und Anlernen des Scalers (d.h. Scaler lernt Mittelwerte und Standardabweichungen der Features)
scaler = StandardScaler().fit(X_train)

# Normierung der Daten mit den gelernten Paramatern und gleichzeitiges Formatieren
X_train = scaler.transform(np.vstack(X_train)).reshape(nr_train, maxlen, -1)
X_val = scaler.transform(np.vstack(X_val)).reshape(nr_val, maxlen, -1)
X_test = scaler.transform(np.vstack(X_test)).reshape(nr_test, maxlen, -1)
print('Datenvorbereitung fertig')

In [ ]:
''' Überprüfen der Form der Numpy Arrays. '''
print('Shape X_train:\t' + str(X_train.shape))
print('Shape X_val:\t' + str(X_val.shape))
print('Shape X_test:\t' + str(X_test.shape))

## 4. Modelle bilden - Dense Modell

### 1.) Dense Modell erstellen

Als Baseline wird zuerst ein Dense Modell erstellt, wessen Performance danach mit dem des RNNs verglichen werden kann.

TODO:
* Importiere den notwendigen Layer
* Zunächst muss die Modellstruktur definiert werden. 
* Die Anzahl der Layer und die Anzahl der Knoten/Nodes pro Layer ist einem selbst überlassen. 
* Probiere also verschiedene Modellgrößen aus, um das beste Modell für die Aufgabe zu finden.

In [ ]:
'''Modellaufbau'''
# Import der notwendigen Klassen und Funktionen
TODO

# Konvertieren der Daten für das Dense Modell
# d.h. Flatten der Daten in 1D Tensoren
X_train_dense = X_train.reshape(nr_train, -1)
X_val_dense = X_val.reshape(nr_val, -1)
X_test_dense = X_test.reshape(nr_test, -1)

''' Überprüfen der Form der Numpy Arrays. '''
print('Shape X_train:\t' + str(X_train_dense.shape))
print('Shape X_val:\t' + str(X_val_dense.shape))
print('Shape X_test:\t' + str(X_test_dense.shape))


# Definition des Input_Shapes
input_shape = (maxlen,)

# Definition des Modells
model=Sequential()
"""
*
* TODO: 
*      Dense Layers hinzufügen
*      Achtung auf passende Output Layer achten!
*
"""

model.summary()

### 2.) Modellkompilierung
TODO: 
* Bestimme, welche Loss Funktion verwendet werden soll. 

In [ ]:
''' Modell kompilieren. '''
# Optimizer festlegen
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)

# Einstellung für das spätere Training zum Kompilieren festlegen
model.compile(optimizer=optimizer, loss=TODO, metrics=['acc'])

### 3.) Trainieren des Dense Modells

TODO:
* Wähle die Anzahl der Epochen aus, für die das Modell trainiert werden soll. 
* Danach müssen die Trainingsinputs, -outputs sowie die Validierungsdaten bestimmt werden.

Ausgabe: 
* Zwischenergebnisse je Epoche

In [ ]:
''' Modell trainieren '''
# Festlegung der Batch_Size und Anzahl der Epoche
batch_size = 32
epochs = TODO

# Durchführen des Trainings. Die Ergebnisse je Epoche werden in der History gespeichert. 
history = model.fit(x=TODO, y=TODO, batch_size=batch_size, epochs=epochs,
                    validation_data=(TODO, TODO), shuffle=True)

## 5. Modelle validieren - Dense Modell

### Kennzahlen fürs Training und Validierung:
Übergeben Sie der Evaluate Methode die Validierungdaten zur Evaluation.

In [ ]:
print('Kennzahlen (Loss und Accuracy Metrik) fürs Training beim Dense Modell:')
print(model.evaluate(X_train_dense, y_train))
print('Kennzahlen (Loss und Accuracy Metrik) für Validierung beim Dense Modell:')
print(model.evaluate(X_val_dense, y_val))

## Visualisierung der Ergebnisse

In [ ]:
""" Trainingsverlauf plotten. """
train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(train_acc) +1)

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_acc,'bo', label='Training accuracy')
plt.plot(epochs, val_acc,'b', label='Validation accuracy')
plt.xlim((0, max(epochs)))
plt.ylim(top=1.0)  # adjust the top leaving bottom unchanged
plt.title('Loss Dense NN', fontsize = 16)
plt.title('Accuracy', fontsize=16)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.xlim((0, max(epochs)))
plt.ylim(bottom=0.0)  
plt.title('Loss Dense NN', fontsize = 16)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## 4. Modelle bilden - Simple RNN

### 1. Modellaufbau - Simple RNN

TODO:
* Importiere die notwendigen Layer
* Definiere die Modellstruktur 
* Die Anzahl der Layer und die Anzahl der Knoten/Nodes pro Layer ist einem selbst überlassen. 
* Probiere verschiedene Modellgrößen aus, um das beste Modell für die Aufgabe zu finden.

Achtung:  
Dieses Problem ist etwas komplexer, daher rate ich hier zu zwei Schichten von SimpleRNN Layern. 

https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN

In [ ]:
''' Modellaufbau '''
# Import der notwendigen Layer
TODO

feature_number = 1
input_shape = (maxlen, feature_number)

"""Model"""

model=Sequential()
"""
*
* TODO: 
*      Dense Layers hinzufügen
*      Achtung auf passende Output Layer achten!
*
"""

model.summary()

### 2.) Modellkompellieren - SimpleRNN 

TODO: 
* Bestimme, welche Loss Funktion verwendet werden soll. 

Optional:  
* Teste unterschiedliche Lernraten oder verwende auch mal einen anderen Optimierer

Hinweise:  
Die Lernrate ist ein sehr wichtiger Hyperparameter, der viel Einfluss auf den Verlauf und Schnelligkeit des Trainings hat!

In [ ]:
''' Modell kompilieren. '''
# Optimizer festlegen
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Einstellung für das spätere Training zum Kompilieren festlegen
model.compile(optimizer=optimizer, loss=TODO, metrics=['acc'])

### 3.) Modelltraining - SimpleRNN

TODO:
* Wähle die Anzahl der Epochen aus, für die das Modell trainiert werden soll. 
* Danach müssen die Trainingsinputs, -outputs sowie die Validierungsdaten bestimmt werden.

Ausgabe: 
* Zwischenergebnisse je Epoche

In [ ]:
''' Modell trainieren '''
# Festlegung der Batch_Size und Anzahl der Epoche
epochs=TODO
batch_size=32

# Durchführen des Trainings. Die Ergebnisse je Epoche werden in der History gespeichert. 
history = model.fit(x=TODO, y=TODO, batch_size=batch_size, epochs=epochs,
                  validation_data=(TODO, TODO))

## 5. Modelle validieren - SimpleRNN

### Kennzahlen fürs Training und Validierung:
TODO:
* Übergebe jeweils die Trainingsdaten bzw. Validierungsdaten dem evaluate Befehl. 


In [ ]:
''' Bewertung des Trainings anhand der Valididierungsdaten mittels der Accuracy. '''
print('Kennzahlen (Loss und Accuracy Metrik) fürs Training:')
print(model.evaluate(TODO, TODO))
print('Kennzahlen (Loss und Accuracy Metrik) für Validierung:')
print(model.evaluate(TODO, TODO))

## Visualisierung der Ergebnisse

In [ ]:
""" Trainingsverlauf plotten. """
train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(train_acc) +1)

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_acc,'bo', label='Training accuracy')
plt.plot(epochs, val_acc,'b', label='Validation accuracy')
plt.xlim((0, max(epochs)))
plt.ylim(top=1.0)  # adjust the top leaving bottom unchanged
plt.title('Accuracy SimpleRNN', fontsize=16)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure(figsize=(12, 8))
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.xlim((0, max(epochs)))
plt.ylim(bottom=0.0)  
plt.title('Loss SimpleRNN', fontsize = 16)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 6. Modell testen & anwenden

Führen Sie diesen Schritt erst aus, wenn Sie mit Ihrem Modell zufrieden sind.  

Der Test eines Modells soll ermitteln, wie gut das Modell übertragbar und generalisierbar ist.  
Wenn Sie alle Modelle direkt testen, wählen Sie nur das Modell aus, was am Besten zum Testdatensatz passt. Aber nicht welches das Problem grundsätzlich am Besten löst. Sie Overfitten somit den Testdatensatz.

TODO:
* Übergebe die Testdaten dem evaluate Befehl zur finalen Bewertung

In [ ]:
''' Bewertung des Trainings anhand der Valididierungsdaten mittels der Accuracy. '''
print('Kennzahlen (Loss und Accuracy Metrik) bei Testen:')
model.evaluate(TODO,TODO)

In [ ]:
"""Modell anhand Testdaten bewerten"""
y_test_pred = (model.predict(X_test)>0.5).astype("int32")
# Berechne Genauigkeit auf den Testdaten
accuracy_test = accuracy_score(y_test, y_test_pred)

# Berechne den F1-Score auf den Testdaten
f1score_test = f1_score(y_test, y_test_pred)

# Ausgabe der Modellgenauigkeit
print('Ergebnis für den Test:')
print('Accuracy: \t' + str(round(accuracy_test, 4)))
print('F1-Score: \t' + str(round(f1score_test, 4)))

# Visualisierung der Konfusionsmatrix
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)
plt.grid()
plt.title('Konfusionsmatrix auf Testdaten')
plt.show()

# Classification Report
print('Classification Report')
print(classification_report(y_test, y_test_pred))

## 7. Bonus Shuffle der Daten
Um mit den geshuffleten (durchgemischte, permutierte) Daten ein neues Modell zu trainieren, führe die nachfolgenden Code-Zellen aus. 
Dann ersetze in der Code-Zelle "Aufteilen in Trainings-, Validierungs und Testdaten." in der 8. Zeile den Ausdruck "X_pre"  durch X_shuffle. Führe danach die Code-Zelle "Aufteilen in Trainings-, Validierungs und Testdaten." und alle folgenden Zellen mit dem Training und Validierung der Modelle aus, um ein neues Modell mit den geshuffelten Daten zu bekommen. 

In [ ]:
""" shuffle cycles with a random permutation """
shuffled_indices = np.random.permutation(maxlen)
X_shuffle = np.array([x[shuffled_indices[shuffled_indices<len(X_pre)]] for x in X_pre], dtype='float32')
print('Zeitschritte zufällig durchmischt')

In [ ]:
''' Visualisierung bevor und nach Durchmischung. '''
run = random.randint(0,155)
tmp_run = X_pre[run]
tmp_run_shuffle = X_shuffle[run]

plt.figure(figsize=(12 ,8))
plt.title('Beschleunigung in X',fontsize = 16)
plt.plot(tmp_run[:][tmp_run[:]!=0],'-ob', label='Run '+str(run))
plt.legend()

plt.figure(figsize=(12, 8))
plt.title('Beschleunigung in X',fontsize = 16)
plt.plot(tmp_run_shuffle[:][tmp_run_shuffle[:]!=0],'-ob', label='shuffled run '+str(run))
plt.legend()
plt.show()